# ✋ **Z punktów do gestów – rozpoznawanie dłoni w akcji!**
Cześć! Miło nam powitać Was na naszym workshopie dotyczącym wizji komputerowej, a konkretnie - klasyfikacji gestów. W tym notatniku postaramy się zaimplementować rozwiązanie, które pozwoli w czasie rzeczywistym odgadywać gest pokazywany przez Ciebie do kamery. Jeśli cokolwiek będzie niejasne, nie wahaj się pytać :)

## **0. Zapoznanie z MediaPipe**
Czas wykryć pierwsze punkty kluczowe! Uruchom plik `data_collection.py`, a rozpocznie się zczytywanie obrazu z Twojej kamery. Możesz włączyć lub wyłączyć dodatkowe wyświetlanie punktów wciskając 'k', a wyjść przy użyciu 'q'.

## **1. Dane**
Aby rozwiązać nasz problem wykorzystując nadzorowane uczenie maszynowe, naturalnie będziemy potrzebowali zbioru oetykietowanych danych. W tym celu zebraliśmy kilkadziesiąt klatek, na których pokazujemy 5 różnych gestów:
- **palm up** ✋
- **fist** 👊
- **peace** ✌
- **thumbs up** 👍
- **ok sign** 👌<br/>

Aby uzyskać do nich dostęp, wczytaj i wyświetl zawartość pliku `data.csv`:

In [1]:
import pandas as pd

url_train = "data_train.csv"
url_test = "data_test.csv"
df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

X_train = df_train.drop("label", axis=1).drop("timestamp", axis=1).values
y_train = df_train["label"].values
X_test = df_test.drop("label", axis=1).drop("timestamp", axis=1).values
y_test = df_test["label"].values

df_train.head(5)

,timestamp,label,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,20250527_220902_239821,0,0.164311,0.917540,4.481203e-07,0.237788,0.867056,-0.037226,0.285385,0.779885,...,-0.012661,0.123909,0.580276,-0.030188,0.128454,0.524540,-0.039614,0.133937,0.472494,-0.045320
1,20250527_220903_380702,0,0.179728,0.908530,3.735631e-07,0.249407,0.842761,-0.019340,0.281542,0.741904,...,-0.027565,0.127473,0.599353,-0.047594,0.129081,0.540624,-0.056885,0.130638,0.483725,-0.062061
2,20250527_220904_041102,0,0.182807,0.899961,4.180846e-07,0.243997,0.831353,-0.018013,0.266820,0.730360,...,-0.035355,0.129937,0.605380,-0.056436,0.131516,0.545716,-0.065307,0.133109,0.488946,-0.069751
3,20250527_220904_977209,0,0.180923,0.890283,3.086925e-07,0.240848,0.807837,0.003075,0.250985,0.699293,...,-0.060574,0.144012,0.617618,-0.085281,0.146537,0.555789,-0.098016,0.149836,0.497431,-0.106504
4,20250527_220906_226695,0,0.177329,0.883633,3.296727e-07,0.234515,0.801488,0.006527,0.236261,0.692671,...,-0.067168,0.160384,0.627052,-0.090832,0.167594,0.567719,-0.103958,0.175674,0.512170,-0.114012


## **2. Model**

Mając przygotowane dane do naszego zadania, możemy przystąpić do trenowania modelu dokonującego klasyfikacji gestów.

Biorąc pod uwagę punkty charaktetystyczne z pojedynczej klatki podczas inferencji, mamy do czynienia z danymi de facto tabelarycznymi, zatem rozsądnym wyborem modelu może być KNN, SVM, Random Forest (bądź inne modele drzewkowe) lub sieć neuronowa MLP.

W poniższej komórce znajdują się importy i wstępne ustawienia potrzebne do użycia każdego z wymienionych modeli.

### Zadanie

Wytrenuj każdy z powyższych modeli i zapisz je do osobnych plików pickle, by móc wykorzystać je później w inferencji przy użyciu skryptu Pythonowego. Zwróć uwagę na różnice w dokładności między modelami oraz wpływ różnych wartości hiperparametrów na wyniki.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import keras
import pickle


models_dict = {
    "KNN": KNeighborsClassifier(),
    "SVM": SVC(),
    "RandomForest": RandomForestClassifier()
}

# kod służący do treningu identyczny dla wszystkich modeli z scikit-learn
print("\nscikit-learn models:")
print("-" * 50)
for name, model in models_dict.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(f"\n🔍 {name} Results:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

# kod służący do treningu sieci neuronowej MLP
print("\n🔮 Neural Network Training:")
print("-" * 50)
model_nn = keras.models.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(len(np.unique(y_train)), activation='softmax')
])

model_nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model_nn.fit(X_train, y_train, epochs=20, validation_split=0.2, verbose=1)

# Ewaluacja sieci neuronowej
loss, acc = model_nn.evaluate(X_test, y_test)
print(f"\n📊 Neural Network Final Results:")
print(f"Test Accuracy: {acc:.2f}")
print(f"Test Loss: {loss:.2f}")

# Zapisz najlepiej działający model
best_model = models_dict["RandomForest"]  # wybierz najlepiej działający model
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print("\n💾 Best model saved as 'best_model.pkl'")


scikit-learn models:
--------------------------------------------------

🔍 KNN Results:
Accuracy: 0.44

Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.40      0.36         5
           1       0.00      0.00      0.00         5
           2       1.00      0.40      0.57         5
           3       0.50      0.60      0.55         5
           4       0.36      0.80      0.50         5

    accuracy                           0.44        25
   macro avg       0.44      0.44      0.40        25
weighted avg       0.44      0.44      0.40        25


🔍 SVM Results:
Accuracy: 0.40

Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.40      0.33         5
           1       0.00      0.00      0.00         5
           2       1.00      0.20      0.33         5
           3       0.60      0.60      0.60         5
           4       0.33      0.80      0.47         5

  

C:\Users\piotr\Desktop\AGH-AI-Days-2025\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\piotr\Desktop\AGH-AI-Days-2025\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\piotr\Desktop\AGH-AI-Days-2025\venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step - accuracy: 0.0000e+00 - loss: 1.8557 - val_accuracy: 1.0000 - val_loss: 1.2806
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.1861 - loss: 1.7505 - val_accuracy: 0.0625 - val_loss: 1.5023
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.4000 - loss: 1.6491 - val_accuracy: 0.0000e+00 - val_loss: 1.6966
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.2292 - loss: 1.6128 - val_accuracy: 0.0000e+00 - val_loss: 1.8180
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.2611 - loss: 1.5551 - val_accuracy: 0.0625 - val_loss: 1.9182
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.4139 - loss: 1.5398 - val_accuracy: 0.0625 - val_loss: 2.0060
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.5097 - loss: 1.5041 - val_accuracy: 0.0625 - val_loss: 2.0485
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2597 - loss: 1.5011 - val_accuracy: 0.0625 - val_